Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 8.58 ms, sys: 65 µs, total: 8.65 ms
Wall time: 12.8 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.1.0.4/7035/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 6 Memory: 7.29 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 271 ms, sys: 13.3 ms, total: 284 ms
Wall time: 2.21 s


[1.8155073941743582,
 1.1481593249285151,
 1.5609626117190065,
 1.8657574379474942,
 1.947850152252406,
 1.4613043889698023,
 1.5605254896030707,
 1.4100673436173627,
 1.4253429493586802,
 1.1565055219371816,
 1.2179877818474842,
 1.3970031257901652,
 1.0966991825329528,
 1.8593313617526341,
 1.2280922078397152,
 1.8630679490763309,
 1.0720730562341467,
 1.2046983630781845,
 1.7541425508262907,
 1.4414686239008252,
 1.9242287574333643,
 1.9198937325425907,
 1.862518485418224,
 1.5507260316013824,
 1.1061833313950693,
 1.8779186052595969,
 1.5328233821626833,
 1.4909598550978536,
 1.7894687612352962,
 1.8951311598287697,
 1.6498412064345314,
 1.2804394449483958,
 1.56808404228066,
 1.041126074782494,
 1.6986519264036979,
 1.322967520565875,
 1.150863974460369,
 1.5866436591788857,
 1.7336163421724762,
 1.914117637499643,
 1.8061140705343197,
 1.5988380070544712,
 1.1326003956941468,
 1.3499788189845403,
 1.7028444523934838]

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.6317282842075067,
 1.2257779719121342,
 1.1868408879157752,
 1.6203666812298643,
 1.859958124909634,
 1.5645482681986134,
 1.4530441512670347,
 1.142152134882239,
 1.0620599068651622,
 1.8364480223920343,
 1.4070483183425764,
 1.1283746174088929,
 1.6010198407590044,
 1.5445102152229542,
 1.8965792883708295,
 1.8988048682932543,
 1.917686352820512,
 1.8200425900504413,
 1.8780545136891365,
 1.55217229019881,
 1.4367256905549803,
 1.817055293530124,
 1.9694295979597358,
 1.5177645388016137,
 1.6495941826437923,
 1.2080520909067045,
 1.3312667781395744,
 1.9772272066587466,
 1.9622905170149572,
 1.41422312706948,
 1.3879502786680509,
 1.0152256150110557,
 1.170782911922902,
 1.2653476067937186,
 1.4355007004165268,
 1.616867903932536,
 1.6594947393860107,
 1.6504896607426829,
 1.0249403043987213,
 1.2613124495267125,
 1.7248388224023459,
 1.371917673167585,
 1.5152175503126362,
 1.180822531903298,
 1.3728122394822782,
 1.87819891306899,
 1.7338100857464442,
 1.3355735052927122,
 1.855

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 219 ms, sys: 39.8 ms, total: 259 ms
Wall time: 2.64 s
